# Team Unicorn

![](http://www.levycreative.com/wp/wp-content/uploads/2015/01/10945748_10152844879967949_5392963414676228818_n.jpg)
## Members
* Jackie Mcguire
* Todd Fox
* Jericho Herrera
* Nicki Lang

In [1]:
import matplotlib.pyplot as plt
import numpy
import pandas as pd
from yahoo_quote_download import yqd
import os
import datetime


In [2]:
stocks = pd.read_csv('techiposince2001.csv')
stocks.columns

Index(['Ticker', 'Name', 'LastSale', 'MarketCap', 'ADR TSO', 'IPOyear',
       'Sector', 'Industry'],
      dtype='object')

In [3]:
stocks.dtypes

Ticker        object
Name          object
LastSale     float64
MarketCap    float64
ADR TSO       object
IPOyear       object
Sector        object
Industry      object
dtype: object

In [4]:
stocks = stocks.loc[stocks['ADR TSO'] == 'n/a']

In [5]:
stocks.MarketCap = stocks.MarketCap.apply(lambda x: round(x/10000))
stocks.MarketCap = stocks.MarketCap.apply(lambda y: '${:,.0f}'.format(y))

In [6]:
stocks=stocks.loc[stocks["IPOyear"]!='n/a']
stocks=stocks.loc[stocks["IPOyear"].astype(int)>=2000]
stocks.head()
stocks_sort = stocks.drop(['ADR TSO', 'LastSale', 'Sector'], axis=1)
stocks_sort

,Ticker,Name,MarketCap,IPOyear,Industry
0,ACMR,"ACM Research, Inc.","$8,916",2017,Industrial Machinery/Components
1,ALTR,Altair Engineering Inc.,"$139,644",2017,Computer Software: Prepackaged Software
3,AYX,"Alteryx, Inc.","$158,743",2017,Computer Software: Prepackaged Software
4,APPN,Appian Corporation,"$125,190",2017,Computer Software: Prepackaged Software
5,AQ,Aquantia Corp.,"$36,264",2017,Semiconductors
7,BAND,Bandwidth Inc.,"$37,939",2017,Computer Software: Prepackaged Software
9,BKI,"Black Knight, Inc.","$691,382",2017,Computer Software: Prepackaged Software
14,CARG,"CarGurus, Inc.","$327,396",2017,EDP Services
15,CARS,Cars.com Inc.,"$191,670",2017,EDP Services
18,CISN,Cision Ltd.,"$143,483",2017,Computer Software: Prepackaged Software


In [7]:
def load_quote(ticker, startdate, enddate):
	print('===', ticker, '===')
	print(yqd.load_yahoo_quote(ticker, startdate, enddate))
	print(yqd.load_yahoo_quote(ticker, startdate, enddate, 'dividend'))
	print(yqd.load_yahoo_quote(ticker, startdate, enddate, 'split'))
def save_quote(ticker, startdate, enddate, filename):
    quotecsv = (yqd.load_yahoo_quote(ticker, startdate, enddate))
    with open(filename, 'a') as myfile:
        header=True
        for row in quotecsv:
            if header == True:
                myfile.write(row+','+'Ticker'+'\n')
                header=False
            else:
                myfile.write(row+','+ticker+'\n')
            
def test():
	# Download quote for stocks
	load_quote(ticker, startdate, enddate)


In [135]:
for row, year in zip(stocks["Ticker"],stocks["IPOyear"]):
    try:
        save_quote(row, str(year)+'0101', str(year)+'1231', 'firstyeardata.csv')
    except:
        pass


In [20]:
firstyear_df = pd.read_csv('firstyeardata.csv')
firstyear_df["Date"] = pd.to_datetime(firstyear_df["Date"], errors='coerce')
def format_epoch(dt):
    return (dt - datetime.datetime(1970,1,1)).total_seconds()
def format_date(sec):
    try:
        return datetime.datetime.fromtimestamp(sec).strftime('%D')
    except:
        return None
firstyear_df["Date"] = firstyear_df["Date"].apply(format_epoch)
firstyear_df

IPOdate_df=firstyear_df.loc[firstyear_df.groupby("Ticker")['Date'].idxmin()]
IPOdate_df["Date"] = IPOdate_df["Date"].apply(format_date)
IPOdate_df
merged_df = pd.merge(stocks_sort, IPOdate_df, on='Ticker', how='outer')
merged_df["Date"] = pd.to_datetime(merged_df["Date"], infer_datetime_format=True)
merged_df['1yrbeg']=merged_df["Date"].apply(lambda x: x+pd.DateOffset(years=1))
merged_df['1yrend']=merged_df["Date"].apply(lambda x: x+pd.DateOffset(years=1, days=3))
merged_df['3yrbeg']=merged_df["Date"].apply(lambda x: x+pd.DateOffset(years=3))
merged_df['3yrend']=merged_df["Date"].apply(lambda x: x+pd.DateOffset(years=3, days=3))
merged_df['5yrbeg']=merged_df["Date"].apply(lambda x: x+pd.DateOffset(years=5))
merged_df['5yrend']=merged_df["Date"].apply(lambda x: x+pd.DateOffset(years=5, days=3))


,Ticker,Name,MarketCap,IPOyear,Industry,Date,Open,High,Low,Close,Adj Close,Volume,1yrbeg,1yrend,3yrbeg,3yrend,5yrbeg,5yrend
0,ACMR,"ACM Research, Inc.","$8,916",2017,Industrial Machinery/Components,2017-10-25,10.230000,10.230000,10.093000,10.110000,10.110000,5800,2018-10-25,2018-10-28,2020-10-25,2020-10-28,2022-10-25,2022-10-28
1,ALTR,Altair Engineering Inc.,"$139,644",2017,Computer Software: Prepackaged Software,2017-10-25,13.230000,13.230000,13.230000,13.230000,13.230000,0,2018-10-25,2018-10-28,2020-10-25,2020-10-28,2022-10-25,2022-10-28
2,AYX,"Alteryx, Inc.","$158,743",2017,Computer Software: Prepackaged Software,2017-03-23,17.250000,17.500000,15.030000,15.500000,15.500000,8660500,2018-03-23,2018-03-26,2020-03-23,2020-03-26,2022-03-23,2022-03-26
3,APPN,Appian Corporation,"$125,190",2017,Computer Software: Prepackaged Software,2017-05-24,15.050000,16.750000,14.600000,15.010000,15.010000,4949248,2018-05-24,2018-05-27,2020-05-24,2020-05-27,2022-05-24,2022-05-27
4,AQ,Aquantia Corp.,"$36,264",2017,Semiconductors,2017-01-02,0.220000,0.220000,0.220000,0.220000,0.220000,0,2018-01-02,2018-01-05,2020-01-02,2020-01-05,2022-01-02,2022-01-05
5,BAND,Bandwidth Inc.,"$37,939",2017,Computer Software: Prepackaged Software,2017-10-25,5.390000,5.390000,5.390000,5.390000,5.390000,0,2018-10-25,2018-10-28,2020-10-25,2020-10-28,2022-10-25,2022-10-28
6,BKI,"Black Knight, Inc.","$691,382",2017,Computer Software: Prepackaged Software,2017-01-02,0.035000,0.035000,0.035000,0.035000,0.035000,0,2018-01-02,2018-01-05,2020-01-02,2020-01-05,2022-01-02,2022-01-05
7,CARG,"CarGurus, Inc.","$327,396",2017,EDP Services,2017-10-11,29.000000,30.190001,27.100000,27.580000,27.580000,12276500,2018-10-11,2018-10-14,2020-10-11,2020-10-14,2022-10-11,2022-10-14
8,CARS,Cars.com Inc.,"$191,670",2017,EDP Services,2017-05-31,25.590000,25.590000,25.590000,25.590000,25.590000,0,2018-05-31,2018-06-03,2020-05-31,2020-06-03,2022-05-31,2022-06-03
9,CISN,Cision Ltd.,"$143,483",2017,Computer Software: Prepackaged Software,2017-06-29,10.050000,10.360000,10.002000,10.290000,10.290000,1433087,2018-06-29,2018-07-02,2020-06-29,2020-07-02,2022-06-29,2022-07-02


,Ticker,Name,MarketCap,IPOyear,Industry,Date,1yr,3yr,5yr
0,ACMR,"ACM Research, Inc.","$8,916",2017,Industrial Machinery/Components,2017-10-26,2018-10-26,2020-10-26,2022-10-26
1,ALTR,Altair Engineering Inc.,"$139,644",2017,Computer Software: Prepackaged Software,2017-10-26,2018-10-26,2020-10-26,2022-10-26
2,AYX,"Alteryx, Inc.","$158,743",2017,Computer Software: Prepackaged Software,2017-03-24,2018-03-24,2020-03-24,2022-03-24
3,APPN,Appian Corporation,"$125,190",2017,Computer Software: Prepackaged Software,2017-05-25,2018-05-25,2020-05-25,2022-05-25
4,AQ,Aquantia Corp.,"$36,264",2017,Semiconductors,2017-01-03,2018-01-03,2020-01-03,2022-01-03
5,BAND,Bandwidth Inc.,"$37,939",2017,Computer Software: Prepackaged Software,2017-10-26,2018-10-26,2020-10-26,2022-10-26
6,BKI,"Black Knight, Inc.","$691,382",2017,Computer Software: Prepackaged Software,2017-01-03,2018-01-03,2020-01-03,2022-01-03
7,CARG,"CarGurus, Inc.","$327,396",2017,EDP Services,2017-10-12,2018-10-12,2020-10-12,2022-10-12
8,CARS,Cars.com Inc.,"$191,670",2017,EDP Services,2017-06-01,2018-06-01,2020-06-01,2022-06-01
9,CISN,Cision Ltd.,"$143,483",2017,Computer Software: Prepackaged Software,2017-06-30,2018-06-30,2020-06-30,2022-06-30


In [23]:
for row, begin, end in zip(merged_df["Ticker"],(merged_df["1yrbeg"]).replace("-",""), (merged_df["1yrend"]).replace("-","")):
    try:
        save_quote(row, begin, end, 'returndata.csv')
    except:
        pass
